In [3]:
using RCall
using Mosek
using JuMP
using CSV

In [4]:
L = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));

In [20]:
# vanilla interior point method for the primal
function vanilla_primal(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, x[1:m] >= 0, start = 1/m);
    @NLobjective(mod, Min, -sum(log(sum(L[i,j]*x[j] for j = 1:m)) for i = 1:n)/n);
    @constraint(mod, sum(x) == 1)
    solve(mod);
    x = getvalue(x);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x))
end

vanilla_primal (generic function with 1 method)

In [27]:
# vanilla interior point method for the primal
function vanilla_boxed(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, x[1:m] >= 0, start = 1/m);
    @NLobjective(mod, Min, -sum(log(sum(L[i,j]*x[j] for j = 1:m)) for i = 1:n)/n + sum(x[i] for i = 1:m));
    solve(mod);
    x = getvalue(x);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x))
end

vanilla_boxed (generic function with 1 method)

In [17]:
# vanilla interior point method for the primal
function vanilla_dual(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, y[1:n] >= 0, start = 1/n);
    @NLobjective(mod, Min, -sum(log(y[i]) for i = 1:n)/n);
    @constraint(mod, ic, L'*y .<= 1)
    solve(mod);
    x = -getdual(ic);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x))
end

vanilla_dual (generic function with 1 method)

In [14]:
function REBayes(L)
    @rput L;
    R"require(REBayes);
    t_rebayes = system.time(res <- KWDual(L, rep(1,dim(L)[2]), rep(1,dim(L)[1])/dim(L)[1]))[3];
    res$f[res$f < 1e-4] = 0
    x_rebayes = res$f / sum(res$f)"
    @rget x_rebayes;
    return sparse(x_rebayes)
end

REBayes (generic function with 1 method)

In [23]:
@time vanilla_dual(L)

  0.297113 seconds (437.38 k allocations: 53.632 MiB, 41.91% gc time)


20-element SparseVector{Float64,Int64} with 3 stored entries:
  [1 ]  =  0.469905
  [13]  =  0.305865
  [14]  =  0.22423

In [29]:
@time vanilla_primal(L)

  0.860932 seconds (1.02 M allocations: 133.443 MiB, 27.82% gc time)


20-element SparseVector{Float64,Int64} with 3 stored entries:
  [1 ]  =  0.469903
  [13]  =  0.305866
  [14]  =  0.224231

In [31]:
@time vanilla_boxed(L)

  0.818472 seconds (1.02 M allocations: 133.433 MiB, 15.06% gc time)


20-element SparseVector{Float64,Int64} with 3 stored entries:
  [1 ]  =  0.469901
  [13]  =  0.305867
  [14]  =  0.224232

In [24]:
@time REBayes(L)

  0.195636 seconds (170 allocations: 6.984 KiB)


20-element SparseVector{Float64,Int64} with 3 stored entries:
  [1 ]  =  0.469905
  [13]  =  0.305865
  [14]  =  0.22423

In [16]:
# L = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));
# warning : the following code might be extremely slow (with this large L)
# @time vanilla_primal(L)
# @time vanilla_dual(L)